# Time series forecasting using Radial basis function network ( regression)

Data structure: 
- ID: Product Identifier
- DATEOP: Date Operation (weekly)
- NBRE: Sales/Demands count/quantity

Goal n°1: one-month-ahead forecasting of each product

Goal n°2: 4-month-ahead forecasting of each product.

"Product" refers to an anonymous banking product.


In [ ]:
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from pandas import DataFrame
from pandas import concat
from matplotlib.pylab import rcParams
from collections import defaultdict
from scipy.spatial import distance
from sklearn.cluster import KMeans
import numpy as np
from sklearn.neural_network import MLPRegressor
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.optimizers import SGD
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import optimizers 


In [ ]:
path=r'C:/Users/yesmi/Desktop/engineerinternship/btseries.txt'
data= pd.read_csv(path,sep=';')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
# 63 products
data.ID.value_counts()

In [ ]:
data.ID=data.ID.astype('category')

In [ ]:
from dateutil import parser
for i in range (data.shape[0]):
    data.DATEOP[i]=parser.parse(data.DATEOP[i])


In [ ]:
#184 dates
data.DATEOP.value_counts()

In [ ]:
data.dtypes

# Convert a Time Series forecasting  to a Supervised Learning task

We have a multivariate time series data.
We are going to use one stop sliding window approach to predict one-month-ahead of each product.

In [ ]:
#First transformation
dat=data.pivot_table(index='DATEOP',
                             columns='ID',
                             values='NBRE',
                             aggfunc='sum')

In [ ]:
dat=pd.DataFrame(dat)
dat.shape

In [ ]:
dat

In [ ]:
Products=dat.columns
Products

In [ ]:
dat=dat.fillna(value=0)

In [ ]:
#second transformation

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
 


In [ ]:
import numpy as np
dat=np.array(dat)
dat

In [ ]:
#we will be using the 10 final dates ( for each product) to predict the value of the next month
dt=series_to_supervised(dat,10,1)

In [ ]:
dt.shape

In [ ]:
dt.head(20)

# Applying RBFN to predict the value of the first product

An example to validate the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( dt.loc[:,'var1(t-10)':'var63(t-1)'] , dt.loc[:,'var1(t)':'var63(t)'] , test_size=0.30, random_state=42)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

In [ ]:
y_train.columns=Products
y_test.columns=Products

In [ ]:
# Gaussian function

def RBFunction(r, x,c ):
    return( np.exp(- (distance.euclidean(x, c))**2/r**2))

# Computing radial/spread

def radial (x,c ):
    r=[]
    for i in range (c.shape[0]):
        k=[]
        for j in range(x.shape[0]) :
            k.append(distance.euclidean(x.iloc[j,:], c[i])) 
        m=(sum(k)/len(k))
        r.append(m)
    return(r)

# compute the centers using Kmeans

def centers (data,n):
    kmeans = KMeans(n_clusters=n)
    a = kmeans.fit(data)
    return(kmeans.cluster_centers_)     

#Preparing the RBF network

def RBF (centers,radials, X): 
    s=[[]for k in range (centers.shape[0])]
    
    for i in range (centers.shape[0]):
        for j in range (X.shape[0]):
            X=pd.DataFrame(X)
            s[i] .append(RBFunction(radials[i] ,X.iloc[j,:],centers[i]))
    s=pd.DataFrame(s)
    return(s.transpose()) 


In [ ]:
# function to evaluate the performance of the models
def evaluate(y,pred):
    #MAPE:Mean absolute percentage error
    def MAPE(y,pred):
        mapev=[]
        aux1=y.reset_index(drop=True)
        aux1=pd.Series(aux1)
        aux2=pd.Series(pred)

        for i in aux1.index:
            if aux1[i]==0:
                mapev.append(0)
            else:
                mapev.append(abs(aux1[i]-aux2[i])/aux1[i])

        mape=(np.mean(mapev))*100
        return(mape)

#MAD:Mean absolute deviation
    def MAD_MSD(y,pred):
        madv=[]
        mapev=[]
        aux1=y.reset_index(drop=True)
        aux1=pd.Series(aux1)
        aux2=pd.Series(pred)
        for i in aux1.index:
            if aux1[i]==0:
                mapev.append(0)
            else:
                mapev.append(abs(aux1[i]-aux2[i])/aux1[i])
        for i in aux1.index:
            if aux1[i]!=0:
               madv.append(mapev[i]*aux1[i])
            else:
               madv.append(0)
        mad=np.mean(madv)
        #MSD:Mean squared deviation
        msdv=np.square(madv)
        msd=np.mean(msdv)
        return(mad,msd)

#SMAPE:Symmetric mean absolute percentage error
    def SMAPE(y,pred):
       smapev=[]
       smape=0
       aux1=y.reset_index(drop=True)
       aux1=pd.Series(aux1)
       aux2=pd.Series(pred)
       for i in aux1.index:
           if aux1[i]==0:
              smapev.append(0)
           else:
               smapev.append(2*(abs(aux1[i]-aux2[i])/((abs(aux1[i])+abs(aux2[i])))))

       smape = (np.mean(smapev))*100
       return(smape)
    MAPE=MAPE(y,pred)
    MAD,MSD=MAD_MSD(y,pred)
    SMAPE=SMAPE(y,pred)
    nn=['MAPE','MAD','MSD','SMAPE']
    kk=[MAPE,MAD,MSD,SMAPE]
    eva=pd.DataFrame([nn,kk])
    return(eva)
    

In [ ]:
#neural network initialization
# Hyperparameter: number of centers/units of the hidden layer nc
nc=80
#Train
c=centers(X_train , nc)
r=radial(X_train,c)
inputs_train= pd.DataFrame(RBF(c,r,X_train))

#Test
inputs_t= pd.DataFrame(RBF(c,r,X_test))

## Starting with linear regression

In [ ]:
#linear model
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
mr=regr.fit(X_train, y_train.iloc[:,0])

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [ ]:
#Test
var=explained_variance_score(y_test.iloc[:,0], y_pred) 
mae=mean_absolute_error(y_test.iloc[:,0], y_pred)
mse=mean_squared_error(y_test.iloc[:,0], y_pred)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], y_pred )  
metric=[var,mae,mse,rmse,r2]
nn=['Explained variance','MAE','MSE','RMSE','R^2']
performance=pd.DataFrame([nn,metric])
performance

In [ ]:
eval1=evaluate( y_test.iloc[:,0],y_pred)
eval1

## Using Sklearn package (MLPRegressor) to implement the second part of the RBFNetwork

In [ ]:
m=MLPRegressor(activation='identity',alpha=0.05,learning_rate ='adaptive',verbose=False)

In [ ]:
mod=m.fit(inputs_train,y_train.iloc[:,0])

In [ ]:
#Train
out=m.predict(inputs_train)
var=explained_variance_score(y_train.iloc[:,0], out) 
mae=mean_absolute_error(y_train.iloc[:,0], out)
mse=mean_squared_error(y_train.iloc[:,0], out)
rmse=np.sqrt(mse)
r2=r2_score(y_train.iloc[:,0], out ) 
#r2a=1-((1-r2)* (1000-1) /(1000-1-5)) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

In [ ]:
evaluate(y_train.iloc[:,0], out)

In [ ]:
#Test
outt=m.predict(inputs_t)
var=explained_variance_score(y_test.iloc[:,0], outt) 
mae=mean_absolute_error(y_test.iloc[:,0], outt)
mse=mean_squared_error(y_test.iloc[:,0], outt)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], outt) 
#r2a=1-((1-r2)* (1000-1) /(1000-1-5)) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

In [ ]:
evaluate(y_test.iloc[:,0], outt)

## Using Kears package to implement the second part of the RBFNetwork

In [ ]:
#def baseline_model():
    # create model
model = Sequential()
model.add(Dense(1, input_dim=nc, kernel_initializer='normal'))
    # Compile model
model.compile(loss='mean_squared_error', optimizer='sgd')

#data preparation
inputs_train_arr=np.array(inputs_train)
inputs_t_arr=np.array(inputs_t)
y_train_arr=np.array(y_train.iloc[:,0])
y_t_arr=np.array(y_test.iloc[:,0])

#train the model
model.fit(inputs_train_arr,y_train_arr, epochs=3000, batch_size=20,verbose=1)

In [ ]:
#Train
out=model.predict(inputs_train_arr)
var=explained_variance_score(y_train.iloc[:,0], out) 
mae=mean_absolute_error(y_train.iloc[:,0], out)
mse=mean_squared_error(y_train.iloc[:,0], out)
rmse=np.sqrt(mse)
r2=r2_score(y_train.iloc[:,0], out ) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

In [ ]:
out=np.array(out)
out=np.squeeze(out)
evaluate(y_train.iloc[:,0], out)

In [ ]:
#Test
outt=model.predict(inputs_t_arr)
var=explained_variance_score(y_test.iloc[:,0], outt) 
mae=mean_absolute_error(y_test.iloc[:,0], outt)
mse=mean_squared_error(y_test.iloc[:,0], outt)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], outt) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

In [ ]:
out=np.array(outt)
out=np.squeeze(out)
evaluate(y_test.iloc[:,0], out)

## Applying RBFN to predict the value of all products for the next month

In [ ]:
#Products= the products' names 
#nc=number of centers
#n_in= how much history
def tspredict1(Products,data,n_in,nc):
   dt=series_to_supervised(data,n_in,1)
   y=pd.DataFrame( dt.loc[:,'var1(t)':])
   X=dt.iloc[:,:(n_in*len(Products))]
   c=centers(X,nc)
   r=radial(X,c)
   inputs= pd.DataFrame(RBF(c,r,X))
   inputs__arr=np.array(inputs)
   y__arr=np.array(y)
   prediction=[]
   performance=pd.DataFrame()
   
   #m=MLPRegressor(activation='identity',alpha=0.05,learning_rate ='adaptive',verbose=False)
 
   for i in range(len(Products)):
        #mod=m.fit(inputs,y.iloc[:,i])
        model.fit(inputs__arr,y__arr[:,i], epochs=100, batch_size=20,verbose=1)
        pred=model.predict(inputs__arr)
        pred=np.squeeze(pred)
        ev=evaluate(y.iloc[:,i],pred)
        prediction.append(pred[len(pred)-1])
        performance=performance.append(ev)
   prediction=pd.DataFrame(prediction)
   return(prediction,performance)

In [ ]:
pred,Performance=tspredict1(Products,dt,10,80)

In [ ]:
pred[pred<0]=0
Products=pd.DataFrame(Products)
Prediction=pd.concat([Products,pred],axis=1)

In [ ]:
#one-month prediction for all products
Prediction

In [ ]:
#Performance for each product prediction
Performance